In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import shapely
import shapely.ops

import pydicom

import pymedphys

In [ ]:
dcm_path = pymedphys.data_path("dedupe-contour.dcm")

In [ ]:
dcm = pydicom.read_file(str(dcm_path), force=True)

In [ ]:
contour_sequences = dcm.ROIContourSequence

In [ ]:
current_contour_sequence = contour_sequences[0]

In [ ]:
current_contour_sequence

In [ ]:
# current_contour_sequence

In [ ]:
contours_by_z = {}
image_sequence_by_z = {}

expected_contour_keys = {
    pydicom.tag.Tag(*tag)
    for tag in [(0x3006, 0x0050), (0x3006, 0x0046), (0x3006, 0x0042), (0x3006, 0x0016)]
}

for contour in current_contour_sequence.ContourSequence:
    if contour.ContourGeometricType != 'CLOSED_PLANAR':
        raise ValueError("Only CLOSED_PLANAR type is supported")
        
    if set(contour.keys()) != expected_contour_keys:
        raise ValueError("Unexpected contour sequence format")
        
    contour_data = contour.ContourData
        
    x = np.array(contour_data[0::3])
    y = np.array(contour_data[1::3])
    z = np.array(contour_data[2::3])
    
    unique_z = np.unique(z)
    
    if len(unique_z) != 1:
        raise ValueError("All z values should be equal")
        
    z = unique_z[0]
    polygon = shapely.geometry.Polygon(zip(x, y))
    
    try:
        contours_by_z[z].append(polygon)
    except KeyError:
        contours_by_z[z] = [polygon]
        
    try:
        image_sequence_by_z[z].append(contour.ContourImageSequence)
    except KeyError:
        image_sequence_by_z[z] = [contour.ContourImageSequence]

In [ ]:
image_sequence_by_z_collapsed = {}

expected_image_sequence_keys = {
    pydicom.tag.Tag(*tag)
    for tag in [(0x0008, 0x1150), (0x0008, 0x1155)]
}

for z, image_sequences in image_sequence_by_z.items():
    values = set()
    
    for image_sequence in image_sequences:
        if len(image_sequence) != 1:
            raise ValueError("Expected only one item per image sequence")
            
        image_sequence_item = image_sequence[0]
        
        if set(image_sequence_item.keys()) != expected_image_sequence_keys:
            raise ValueError("Unexpected contour image sequence format")
            
        values.add((
            image_sequence_item.ReferencedSOPClassUID,
            image_sequence_item.ReferencedSOPInstanceUID,
        ))
    
    if len(values) != 1:
        raise ValueError("Each z value should only point to one image slice")
        
    image_sequence_by_z_collapsed[z] = image_sequences[0]

In [ ]:
merged = {}

for z, contours in contours_by_z.items():
    merged[z] = shapely.ops.unary_union(contours)

In [ ]:
merged[-100]

In [ ]:
new_contour_sequence = pydicom.sequence.Sequence()



In [ ]:
contour_data = contour_sequences[0].ContourSequence[0].ContourData

In [ ]:
x = np.array(contour_data[0::3])
y = np.array(contour_data[1::3])
z = np.array(contour_data[2::3])

z

In [ ]:
assert len(np.unique(z)) == 1

In [ ]:
x

In [ ]:
y

In [ ]:
plt.plot(x, y)
plt.axis('equal')

In [ ]:
shapely.geometry.Polygon(zip(x, y))